# Set-up

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import VGG19
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import itertools

import scipy.io 
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage import rotate


In [ ]:
def show_img(image):
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.show()

IMAG_SIZE = 224 # All images will be resized to 224x224
def normalize_img(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMAG_SIZE, IMAG_SIZE))
  return  image, label

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    name="oxford_flowers102",
    split=['train', 'validation', 'test'],
    with_info=True,
    as_supervised=True,
    batch_size=32
)

train = raw_train.map(normalize_img)
validation = raw_validation.map(normalize_img)
test = raw_test.map(normalize_img)

print(metadata.splits["train"].num_examples)
print(metadata.splits["validation"].num_examples)
print(metadata.splits["test"].num_examples)

# Model Training

In [ ]:
NUM_CLASSES = 102
input_shape = (224,224,3)

pixels = 224
IMAGE_SIZE = (pixels, pixels)

# below you can see the two models we use transfer learning on.
# the first is a version of mobileNet and second resNet50.
# for both we create a model and continue training them on the flowers102 dataset
"""
#################################################################################################
    MobileNet
#################################################################################################
"""

handle_base = "mobilenet_v2_100_224"
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)

mobileNet_fine_tuning = True
mobilenet = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, trainable=mobileNet_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=metadata.features["label"].num_classes)
])
mobilenet.build((None,)+IMAGE_SIZE+(3,))
mobilenet.summary()

"""
#################################################################################################
    ResNet
#################################################################################################
"""

resNet_fine_tuning = True
ResNet_model = keras.Sequential([
    ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet',input_shape=input_shape),
    layers.Dense(30, activation = 'relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=metadata.features["label"].num_classes)
])
ResNet_model.layers[0].trainable = True
ResNet_model.build((None,)+IMAGE_SIZE+(3,))
ResNet_model.summary()

In [ ]:
BATCH_SIZE = 16
EPOCHS = 40

steps_per_epoch = metadata.splits["train"].num_examples / BATCH_SIZE 
validation_steps = metadata.splits["validation"].num_examples / BATCH_SIZE

# Unbatch datasets to avoid batch mismatch
train = train.unbatch().batch(BATCH_SIZE,drop_remainder=True).repeat().shuffle(512)
validation = validation.unbatch().batch(BATCH_SIZE,drop_remainder=True).repeat()

In [ ]:
mobilenet.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=.5e-3), 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

mobilenet.fit(
    x=train,
    epochs=EPOCHS, 
    steps_per_epoch=steps_per_epoch,
    validation_data=validation,
    validation_steps=validation_steps,
    verbose=2).history

mobileNet_score = mobilenet.evaluate(x=test)
print("")
print("Test loss:", mobileNet_score[0])
print("Test accuracy:", mobileNet_score[1])

In [ ]:
mobilenet.save("mobileNet_model")

In [ ]:
ResNet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.5e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=["accuracy"])

ResNet_model.fit(
    x=train,
    epochs=EPOCHS, 
    steps_per_epoch=steps_per_epoch,
    validation_data=validation,
    validation_steps=validation_steps,
    verbose=2).history

ResNet_score = ResNet_model.evaluate(x=test, verbose=0)

print("")
print("Test loss:", ResNet_score[0])
print("Test accuracy:", ResNet_score[1])

In [ ]:
# Saving the model 
ResNet_model.save("resNet_model")